## CNN for Fossil Classification

In [12]:
# Import dependencies
import numpy as np 
import keras
from keras import layers
import tensorflow_datasets as tfds
from tensorflow import data as tf_data
import matplotlib.pyplot as plt

We will use transfer learning using different pre-trained models on the imagenet dataset. We first train the Xception model. 

In [ ]:
# Our images are already resized, but we could include optional resizing in the final model
# We start with data augmentation
augmentation_layers = [
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.25)
]

def data_augmentation(x):
    for layer in augmentation_layers:
        x = layer(x)
        return x

# Create augmented data - Check with Lucy about what we want to do here
augmented_data = training_data.map(lambda x, y: (data_augmentation(x), y))
training_data = training_data.concatenate(augmented_data)

NameError: name 'train_ds' is not defined

In [ ]:
# Batch and prefetch data ?

batch_size = 64 # this could change

training_data = training_data.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()
validation_data = validation_data.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()
test_data = test_data.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()

In [6]:
# We will use transfer learning using different pre-trained models on the imagenet dataset

# Create the base model with the pre-trained imagenet weights
base_model = keras.applications.Xception(
    weights = 'imagenet',
    input_shape = (200, 200, 3),
    include_top = False) # Why do we not include imagenet at the top? 

# Freeze the base model weights (they will not be trained)

base_model.trainable = False

2026-03-01 16:04:44.968352: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Create a new model that goes on top of the base imagenet model
inputs = keras.Input(shape = (200, 200, 3))
# We should scale our inputs to be between -1 and 1 (or should we normalize to have mean 0 and std 1?)
scale_layer = keras.layers.Rescaling(scale = 1/127.5, offset = -1)
x = scale_layer(inputs)

# We want to keep batchnorm layers of base model in inference (predict) mode so we do not train of them. 
# Make sure that base model is running on inference mode

x = base_model(x, training = False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.1)(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary(show_trainable = True)
# I think we should play with the layers we use here. We should try the setup they have, but maybe add more or different layers depending on how well the model does. 

In [ ]:
# Train the new top layer of the model
model.compile(
    optimizer = keras.optimizers.Adam(), # play with this also
    loss = keras.losses.BinaryCrossentropy(from_logits = True), #check this also
    metrics = [keras.metrics.BinaryAccuracy()],
)

epochs = 2 # also play with this
print("Fitting top layer of model")
model.fit(training_data, epochs = epochs, validation_data = validation_data)

In [ ]:
# Fine tune entire model
base_model.trainable = True
model.summary(show_trainable=True)

model.compile(
    optimizer = keras.optimizers.Adam(1e-5), # can also play with learning rate
    loss = keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()] #check this -- we have a binary model so it should be ok
)

epochs = 1 #??
print("Fitting end-to-end model")
model.fit(training_data, epochs = epochs, validation_data = validation_data)

In [ ]:
# Validate model on test data
print("Test dataset evaluation:")
model.evaluate(testing_data)